In [1]:
import subprocess, json

BASE = "http://127.0.0.1:18080"

def call_api(path: str, method: str = "GET", body: dict | None = None):
    url = f"{BASE}{path if path.startswith('/') else '/' + path}"
    args = ["curl", "-s", "-X", method, url, "-H", "Content-Type: application/json"]
    if body is not None:
        args += ["-d", json.dumps(body)]
    result = subprocess.run(args, capture_output=True, text=True)
    raw = result.stdout.strip()
    try:
        print(json.dumps(json.loads(raw), indent=2))
    except json.JSONDecodeError:
        print(raw)


In [2]:
call_api("/health")

{
  "status": "ok"
}


In [3]:
call_api("/db-check")

{
  "ok": true,
  "db_time": "2025-08-31T22:14:07.332052+02:00"
}


In [10]:
call_api("/servers")

[
  {
    "id": 1,
    "hostname": "myserver.local.lan",
    "configuration": {},
    "datacenter_id": 1,
    "created_at": "2024-05-28T11:39:01.516030",
    "modified_at": "2024-05-28T11:39:01.516030"
  },
  {
    "id": 2,
    "hostname": "database.local.lan",
    "configuration": {
      "user_limit": 10,
      "max_connections": 500
    },
    "datacenter_id": 2,
    "created_at": "2024-05-28T11:40:13.349528",
    "modified_at": "2024-05-28T11:40:13.349528"
  },
  {
    "id": 3,
    "hostname": "rabbitmq.local.lan",
    "configuration": {
      "max_queues": 1234
    },
    "datacenter_id": 2,
    "created_at": "2024-05-28T11:41:14.626643",
    "modified_at": "2024-05-28T11:41:14.626643"
  },
  {
    "id": 4,
    "hostname": "db2.local.lan",
    "configuration": {
      "user_limit": 999,
      "max_connections": 900
    },
    "datacenter_id": 1,
    "created_at": "2025-08-30T20:09:40.245498",
    "modified_at": "2025-08-31T20:15:00.962519"
  },
  {
    "id": 9,
    "hostname": "ap

In [5]:
call_api("/servers?datacenter_id=2&sort=hostname")

[
  {
    "id": 2,
    "hostname": "database.local.lan",
    "configuration": {
      "user_limit": 10,
      "max_connections": 500
    },
    "datacenter_id": 2,
    "created_at": "2024-05-28T11:40:13.349528",
    "modified_at": "2024-05-28T11:40:13.349528"
  },
  {
    "id": 3,
    "hostname": "rabbitmq.local.lan",
    "configuration": {
      "max_queues": 1234
    },
    "datacenter_id": 2,
    "created_at": "2024-05-28T11:41:14.626643",
    "modified_at": "2024-05-28T11:41:14.626643"
  }
]


In [6]:
call_api("/servers/2")

{
  "id": 2,
  "hostname": "database.local.lan",
  "configuration": {
    "user_limit": 10,
    "max_connections": 500
  },
  "datacenter_id": 2,
  "created_at": "2024-05-28T11:40:13.349528",
  "modified_at": "2024-05-28T11:40:13.349528"
}


In [7]:
# create new record
call_api("/servers", method="POST", body={
    "hostname": "api1.local.lan",
    "datacenter_id": 1,
    "configuration": {"user_limit": 20}
})

{
  "id": 9,
  "hostname": "api1.local.lan",
  "configuration": {
    "user_limit": 20
  },
  "datacenter_id": 1,
  "created_at": "2025-08-31T20:14:22.982573",
  "modified_at": "2025-08-31T20:14:22.982573"
}


In [9]:
# Update just configuration
call_api("/servers/4", method="PUT", body={
    "configuration": {"user_limit": 999, "max_connections": 900}
})

# Update hostname and datacenter
# call_api("/servers/4", method="PUT", body={
#     "hostname": "db2.local.lan",
#     "datacenter_id": 1
# })

# No fields (should 400)
# call_api("/servers/2", method="PUT", body={})

# Non-existent id (should 404)
# call_api("/servers/9999", method="PUT", body={"hostname": "ghost.local.lan"})

{
  "id": 4,
  "hostname": "db2.local.lan",
  "configuration": {
    "user_limit": 999,
    "max_connections": 900
  },
  "datacenter_id": 1,
  "created_at": "2025-08-30T20:09:40.245498",
  "modified_at": "2025-08-31T20:15:00.962519"
}


In [11]:
# OK delete (pick an ID that isn't linked in switch_to_server)
call_api("/servers/9", method="DELETE")

# Deleting a non-existent ID -> 404
# call_api("/servers/9999", method="DELETE")

# If the server is linked to a switch, expect 400 (FK error)
# e.g., if 3 is linked in sample data:
# call_api("/servers/3", method="DELETE")

{
  "deleted_id": 9
}
